<h1><center>GPU Computing with CuPy</center></h1>

In [ ]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
# cp.cuda.set_allocator(None) # You can disable CuPy memory pool with this 
%load_ext Cython

In [ ]:
from contextlib import contextmanager
from time import time

@contextmanager
def cupy_timer():
    start = cp.cuda.Event()
    end = cp.cuda.Event()
    start.record()
    yield
    end.record()
    end.synchronize()
    elapsed_time = cp.cuda.get_elapsed_time(start, end)
    print(f'Elapsed time: {elapsed_time} ms')
    

@contextmanager
def cpu_timer():
    start = time()
    yield
    end = time()
    print(f'Elapsed time: {(end - start) * 1000} ms')

<h2><center>Matrix Multiplication NumPy vs CuPy</center></h2>

In [ ]:
x_cpu = np.random.random((1000, 1000))
y_cpu = np.random.random((1000, 1000))
x_gpu = cp.array(x_cpu)
y_gpu = cp.array(y_cpu)

with cpu_timer():
    z_cpu = x_cpu @ y_cpu

with cupy_timer():
    z_gpu = x_gpu @ y_gpu

<h2><center>Linear System Solving NumPy vs CuPy</center></h2>

In [ ]:
A_cpu = np.random.random((1000, 1000))
b_cpu = np.random.random(1000)
A_gpu = cp.array(A_cpu)
b_gpu = cp.array(b_cpu)
with cpu_timer():
    np.linalg.solve(A_cpu, b_cpu)
    
with cupy_timer():
    cp.linalg.solve(A_gpu, b_gpu)

<h2><center>Euclidean distance matrix</center></h2>

<center>
$
    d_e(\mathbf x, \mathbf y) =
    \begin{bmatrix}
    \sum_{i=1}^n (x_{1i}-y_{1i})^2 & \sum_{i=1}^n(x_{1i}-y_{2i})^2 & \cdots & \sum_{i=1}^n (x_{1i}-y_{ni})^2 \\  
    \sum_{i=1}^n(x_{2i}-y_{1i})^2 & \sum_{i=1}^n(x_{2i}-y_{2i})^2 & \cdots & \sum_{i=1}^n(x_{2i}-y_{ni})^2 \\  
    \vdots & \vdots & \ddots & \vdots \\
    \sum_{i=1}^n(x_{ni}-y_{1i})^2 & \sum_{i=1}^n(x_{ni}-y_{2i})^2 & \cdots & \sum_{i=1}^n(x_{ni}-y_{ni})^2 \\  
    \end{bmatrix}
$
<bf>
</center>
<h2><center>Vectorization friendly summation<center></h2>
<center>
$ 
\sum_{k=1}^n \left(x_{ik}-y_{ik}\right)^2 = \left(\vec{x_i} - \vec {y_j}\right)\cdot \left(\vec{x_i} - \vec{y_j}\right)=\vec{x_i} \cdot \vec{x_i} + \vec{y_j} \cdot \vec{y_j} \cdot \vec{y_j}-2\vec{x_i}\cdot \vec{y_j}$
</center>

In [ ]:
def euclidean_distance_cpu(x, y):
    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[:, np.newaxis].T
    xy = x @ y.T
    return np.abs(x2 + y2 - 2.0 * xy)

In [ ]:
def euclidean_distance_gpu(x, y):
    x2 = cp.einsum('ij,ij->i', x, x)[:, cp.newaxis]
    y2 = cp.einsum('ij,ij->i', y, y)[:, cp.newaxis].T
    xy = x @ y.T
    return cp.abs(x2 + y2 - 2.0 * xy)

In [ ]:
x_cpu = np.random.random((5000, 5000))
y_cpu = np.random.random((5000, 5000))
x_gpu = cp.array(x_cpu)
y_gpu = cp.array(y_cpu)

with cpu_timer():
    eu_cpu = euclidean_distance_cpu(x_cpu, y_cpu)

with cupy_timer():
    eu_gpu = euclidean_distance_gpu(x_gpu, y_gpu)
    
print(np.allclose(eu_cpu, eu_gpu.get()))

<h2><center>Make function work for both CuPy/NumPy</center></h2>

In [ ]:
def euclidean_distance(x, y):
    modp = cp.get_array_module(x)
    x2 = modp.einsum('ij,ij->i', x, x)[:, modp.newaxis]
    y2 = modp.einsum('ij,ij->i', y, y)[:, modp.newaxis].T
    xy = x @ y.T
    return modp.abs(x2 + y2 - 2.0 * xy)

In [ ]:
with cpu_timer():
    eu_cpu = euclidean_distance(x_cpu, y_cpu)

with cupy_timer():
    eu_gpu = euclidean_distance(x_gpu, y_gpu)
    
print(np.allclose(eu_cpu, eu_gpu.get()))

<h2><center>Calculating/Plotting Julia Sets</center></h2>

In [ ]:
X, Y = np.meshgrid(np.linspace(-2.0 , 2.0, 5000), np.linspace(-2.0, 2.0, 5000))
Z_cpu = X + Y * 1j
Z_gpu = cp.array(Z_cpu)

In [ ]:
def julia_set_naive(Z, c, iter_max):
    julia = np.zeros_like(Z, dtype=np.int32)
    radius = max(np.abs(c), 2.0)
    nx, ny = Z.shape
    for i in range(nx):
        for j in range(ny):
            z = Z[i, j]
            k = 0
            while np.abs(z) < radius and k < iter_max:
                z = z ** 2 + c
                k += 1
            julia[i, j] = k
    return julia

In [ ]:
%%cython -a

# distutils: extra_compile_args = -fopenmp -march=native
# distutils: extra_link_args = -fopenmp
from cython cimport boundscheck, wraparound
from cython.parallel cimport prange

@boundscheck(False)
@wraparound(False)
def julia_set_cython(const double complex[:, :] Z, const double complex c,
                       const int iter_max, const double radius, 
                       int[:, :] julia):
    cdef:
        int i, j, k, nx, ny
        double complex z
    nx = Z.shape[0]
    ny = Z.shape[1]
    for i in prange(nx, nogil=True, schedule='static'):
        for j in range(ny):
            z = Z[i, j]
            k = 0
            while abs(z) < radius and k < iter_max:
                z = z * z + c
                k = k + 1
                
            julia[i, j] = k 

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
julia = np.zeros_like(Z_cpu, dtype=np.int32)
c = -0.9 + 0.22143j
radius = max(abs(c), 2.0)
with cpu_timer():
    julia_set_cython(Z_cpu, c, 1000, radius, julia)
ax.set_aspect('equal')
ax.imshow(julia, extent=[-2, 2, -2, 2]);

In [ ]:
julia_kernel = cp.ElementwiseKernel('complex128 z, complex128 c, int32 itermax, float64 radius',
                                    'int32 julia',
                                    f'''julia = 0;
                                    complex<double> zz = z;
                                    int nit = 0;
                                    while(abs(zz) < radius && nit < itermax) {{
                                        zz = zz * zz + c;
                                        nit += 1;
                                    }}
                                    julia = nit;''', 'julia_kernel')

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
c = -0.9 + 0.22143j
with cupy_timer():
    Z_gpu = cp.array(Z_cpu)
    julia_gpu = julia_kernel(Z_gpu, c, 1000, radius)
    julia_array = julia_gpu.get()
ax.set_aspect('equal')
ax.imshow(julia_array, extent=[-2, 2, -2, 2]);

In [ ]:
julia.sum(), julia_gpu.sum() # Compute sum to validate

<h2><center>CuPy Memory Pools</center></h2>

In [ ]:
!nvidia-smi #

In [ ]:
%whos 

In [ ]:
mempool = cp.get_default_memory_pool()

In [ ]:
print(mempool.used_bytes(), mempool.total_bytes())